<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Description</dt> <dd> Bokeh streams quickstart guide</dd>
  <dt>Author</dt> <dd>Jean-Luc Stevens</dd>
  <dt>HoloViews</dt> <dd>>1.6.2</dd>
  <dt>Python</dt> <dd>2.7/3.3+</dd>
</dl>
</div>

This quickstart is designed to introduce the basics of the streams system interacts with the [Bokeh](http://bokeh.pydata.org/) backend in HoloViews 1.7+. This quickstart guide focuses on stream functionality specific to the Bokeh backend and builds on the introductory [streams quickstart](quickstart/streams.ipynb) notebook. This notebook uses core HoloViews together with the bokeh backend and shows how you can build visualizations that you can directly interact with. To install bokeh with conda, use:

```bash
conda install bokeh
```

In [ ]:
import holoviews as hv
import numpy as np
from holoviews import streams
hv.notebook_extension('bokeh')

# Direct interaction

Unlike matplotlib, the [Bokeh](http://bokeh.pydata.org/) plotting library is designed to support interactive visualizations in the browser. This makes it possible for HoloViews to build Bokeh plots that communicate back to Python from the browser via the stream system allowing direct interaction with your visualizations.

As soon as the bokeh backend is enabled, direct interaction is also enabled. This means that the no new concepts need to be introduced relative to the [streams quickstart](quickstart/streams.ipynb). For instance, here is a simple example using the ``PositionXY`` stream which is already Bokeh enabled:

In [ ]:
def crosshairs(x,y):
    print "x: %s y:%s" %(x,y)
    return hv.HLine(y) * hv.VLine(x) * hv.Ellipse(0,0, 1)

dmap = hv.DynamicMap(crosshairs, kdims=[], streams=[streams.PositionXY(x=0,y=0)])
dmap

Now when you hover over the plot, the crosshair will follow your mouse!

Note that on initialization, we see the initial print message showing that the stream is initialized with ``x=0, y=0``. As before, you can use the ``event`` method on the ``DynamicMap`` which also shows the print message and updates the visualization above:

In [ ]:
dmap.event(x=0.2,y=0.1)

You may be wondering what happens to the print messages when you use the mouse to move the crosshair: the callback is being executed but no print messages are appearing in the notebook. This is because when interacting with the stream in the browser, the callback output is only used to update the plot and there is no other mechanism to update the notebook document. 

These print messages do not vanish entirely: if you go to your browser console you will see that the print output is converted into logging messages in JavaScript. This can help with debugging interaction but it is often simpler to supply explicit values using the ``event`` method which does allow print output to appear in the notebook:

In [ ]:
dmap.event(x=0.3, y=0.3)

## Bokeh tools

Some of the streams are tied to [Bokeh tools](http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html) which means that the events that drive the stream are only generated when the appropriate Bokeh tool is active.

For instance, in the following example, you need to select the 'Box Select' tool from the toolbar on the right before you can draw new ``Bounds`` objects at the position you want:

In [ ]:
dmap = hv.DynamicMap(lambda bounds: hv.Bounds(bounds, extents=(-5, -5, 5, 5)),
              kdims=[], streams=[streams.Bounds(bounds=(-1,-1,1,1))])
dmap

As before you can use the ``event`` method to update the stream parameter value:

In [ ]:
dmap.event(bounds=(-1,-3,1,3))

## Setting the source

The source of the events used to update the stream correspond to the portion of the visualization that corresponds to the ``DynamicMap`` itself. However, this does not have to be the case and you can receive events from any part of the visualization, including from portions that correspond to simple elements.

In the following example, an ``Image`` called ``img`` is declared and the position of the cursor over this image is what sets the values in the ``PositionXY`` stream, even though it is used only by the ``DynamicMap`` on the right side of the ``Layout``. Note that ``img`` is set as the ``source`` of the ``PositionXY`` stream:

In [ ]:
img = hv.Image(np.random.rand(10,10))
extents = (-.5, -.5, .5, .5)
img + hv.DynamicMap(lambda x, y: hv.HLine(y, extents=extents) * hv.VLine(x, extents=extents),
              kdims=[], streams=[streams.PositionXY(source=img)])

Now when you hover over the image on the left, the position of the crosshair on the right is updated.